In [59]:
import pandas as pd
from pandas import read_csv, DataFrame

data = read_csv('ChEC-LEQ_all_recoded_20200114.csv', index_col=None)
data.describe()

,ID,serious_accident,wit_serious_accident,nat_disaster,other_illnessinjury,death_person,death_pet,med_procedures,separation,suicide,...,oth_pos_ageY2,oth_pos_ageM2,oth_pos_ageY3,oth_pos_ageM3,oth_pos_ageY4,oth_pos_ageM4,oth_pos_imp1,oth_pos_imp2,oth_pos_imp3,oth_pos_imp4
count,58.000000,58.000000,58.0,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,...,6.000000,6.000000,3.000000,3.000000,2.000000,2.000000,26.000000,7.000000,3.000000,2.000000
mean,1062.517241,0.017241,0.0,0.0,0.189655,0.241379,0.500000,0.275862,0.120690,0.017241,...,3.500000,6.666667,4.333333,3.333333,4.500000,3.500000,2.307692,2.142857,2.666667,2.500000
std,259.435571,0.131306,0.0,0.0,0.395452,0.431657,0.504367,0.450851,0.328611,0.131306,...,1.643168,4.802777,0.577350,5.773503,0.707107,3.535534,0.837579,0.690066,0.577350,0.707107
min,1000.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,4.000000,0.000000,4.000000,1.000000,0.000000,1.000000,2.000000,2.000000
25%,1014.250000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,3.000000,4.000000,0.000000,4.250000,2.250000,2.000000,2.000000,2.500000,2.250000
50%,1028.500000,0.000000,0.0,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,3.500000,8.000000,4.000000,0.000000,4.500000,3.500000,2.500000,2.000000,3.000000,2.500000
75%,1044.250000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,4.000000,10.750000,4.500000,5.000000,4.750000,4.750000,3.000000,2.500000,3.000000,2.750000
max,3000.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,6.000000,11.000000,5.000000,10.000000,5.000000,6.000000,3.000000,3.000000,3.000000,3.000000


In [60]:
prefixes = ['ser_acc','wit_ser_acc','nat_dis','oth_ill','death_oth','death_pet','med_pro','sep',
            'suicide','assault','threat','kidnap','an_att','wit_fam_vio','wit_fam_threat',
            'wit_oth_vio','yell','lack_care','new_sib','new_scho','new_home','change_fam','hobby',
            'new_friend','end_friend','finan','recog','oth_stress','oth_pos']
neg_pref = ['ser_acc','wit_ser_acc','nat_dis','oth_ill','death_oth','death_pet','med_pro','sep',
            'suicide','assault','threat','kidnap','an_att','wit_fam_vio','wit_fam_threat',
            'wit_oth_vio','yell','lack_care','oth_stress']
pos_pref = ['hobby','new_friend','recog','oth_pos']
ambig_pref = ['new_sib','new_scho','new_home','change_fam','end_friend','finan']

In [61]:
import numpy as np

data['LEQ_tot_neg'] = 0
data['LEQ_sum_sev_neg'] = 0
data['LEQ_tot_pos'] = 0
data['LEQ_sum_sev_pos'] = 0
data['LEQ_tot_endorsed'] = 0

# loop through and start making summary codes
for e in prefixes:
    data[e+'_num_neg_events'] = 0
    data[e+'_num_pos_events'] = 0
    data[e+'_tot_endorsed'] = 0
    data[e+'_num_neg_events'] = (data[e+'_imp1']<0)*1+(data[e+'_imp2']<0)*1+(data[e+'_imp3']<0)*1+(data[e+'_imp4']<0)*1
    data[e+'_num_pos_events'] = (data[e+'_imp1']>0)*1+(data[e+'_imp2']>0)*1+(data[e+'_imp3']>0)*1+(data[e+'_imp4']>0)*1
    data[e+'_tot_endorsed'] = (np.isfinite(data[e+'_imp1']))*1+(np.isfinite(data[e+'_imp2']))*1+(np.isfinite(data[e+'_imp3']))*1+(np.isfinite(data[e+'_imp4']))*1
    negtemp = DataFrame(columns=['_imp1','_imp2','_imp3','_imp4'], index=data.index)
    postemp = DataFrame(columns=['_imp1','_imp2','_imp3','_imp4'], index=data.index)
    for i in ['_imp1','_imp2','_imp3','_imp4']:
        negtemp[i] = abs(data[data[e+i]<0][e+i])
        postemp[i] = data[data[e+i]>0][e+i]
    negtemp.fillna(0,inplace=True)
    postemp.fillna(0, inplace=True)

    data[e+'_sum_sev_neg'] = negtemp.sum(axis=1)
    data[e+'_sum_sev_pos'] = postemp.sum(axis=1)

    data['LEQ_tot_neg'] = data['LEQ_tot_neg'] + data[e+'_num_neg_events']
    data['LEQ_sum_sev_neg'] = data['LEQ_sum_sev_neg'] + data[e+'_sum_sev_neg']
    data['LEQ_tot_pos'] = data['LEQ_tot_pos'] + data[e+'_num_pos_events']
    data['LEQ_sum_sev_pos'] = data['LEQ_sum_sev_pos'] + data[e+'_sum_sev_pos']
    data['LEQ_tot_endorsed'] = data['LEQ_tot_endorsed'] + data[e+'_tot_endorsed']
    
                                                                                                                                           

In [62]:
data.to_csv('ChEC-LEQ_all_scored_20200115.csv')